# Imports

In [1]:
!pip install moviepy

    100% |████████████████████████████████| 378kB 5.7MB/s 
    100% |████████████████████████████████| 3.3MB 7.6MB/s 
    100% |████████████████████████████████| 51kB 16.8MB/s 
  Running setup.py bdist_wheel for moviepy ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/ad/92/4d/a6c6307d4c2219d002646bd4a5987e31fd5697f6ea7778b2c0
Successfully built moviepy


In [3]:
!git clone http://github.com/anantyash9/wip.git

Cloning into 'wip'...
remote: Counting objects: 220, done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 217
Receiving objects: 100% (220/220), 266.35 MiB | 14.82 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Checking out files: 100% (244/244), done.


In [4]:
cd wip

/content/wip


In [5]:
!git clone http://github.com/anantyash9/colab.git

Cloning into 'colab'...
remote: Counting objects: 7288, done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 7288 (delta 6), reused 7 (delta 1), pack-reused 7273
Receiving objects: 100% (7288/7288), 275.87 MiB | 36.10 MiB/s, done.
Resolving deltas: 100% (251/251), done.
Checking out files: 100% (11721/11721), done.


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import glob
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Activation, Cropping2D, MaxPooling2D, Dropout, Reshape, \
    Convolution2D
from moviepy.editor import VideoFileClip
from object_detection.utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util
import json

## Visualization utils

In [0]:
import collections
import functools

_TITLE_LEFT_MARGIN = 10
_TITLE_TOP_MARGIN = 10
STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque',
    'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan',
    'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod',
    'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue',
    'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue',
    'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid',
    'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin',
    'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed',
    'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown',
    'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow',
    'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat', 'White',
    'WhiteSmoke', 'Yellow', 'YellowGreen']
def visualize_boxes_and_labels_on_image_array(
    image,
    boxes,
    classes,classes1,
    scores,scores1,
    category_index,category_index1,
    instance_masks=None,
    instance_boundaries=None,
    keypoints=None,
    use_normalized_coordinates=False,
    max_boxes_to_draw=20,
    min_score_thresh=.5,
    agnostic_mode=False,
    line_thickness=4,
    groundtruth_box_visualization_color='black',
    skip_scores=False,
    skip_labels=False):
  
  # Create a display string (and color) for every box location, group any boxes
  # that correspond to the same location.
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_instance_masks_map = {}
  box_to_instance_boundaries_map = {}
  box_to_keypoints_map = collections.defaultdict(list)
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      if instance_masks is not None:
        box_to_instance_masks_map[box] = instance_masks[i]
      if instance_boundaries is not None:
        box_to_instance_boundaries_map[box] = instance_boundaries[i]
      if keypoints is not None:
        box_to_keypoints_map[box].extend(keypoints[i])
      if scores is None:
        box_to_color_map[box] = groundtruth_box_visualization_color
      else:
        display_str = ''
        if not skip_labels:
          if not agnostic_mode:
            if classes[i] in category_index.keys():
              class_name = category_index[classes[i]]['name']
              class_name1 = category_index1[classes1[i]]['name']
            else:
              class_name = 'N/A'
            display_str = str(class_name)
        if not skip_scores:
          if not display_str:
            display_str = '{}%'.format(int(100*scores[i]))
          else:
            display_str = '{}: {}%'.format(class_name, int(100*scores[i]))
            display_str +=' :: '
            display_str += '{}: {}%'.format(class_name1, int(100*scores1[i]))
        box_to_display_str_map[box].append(display_str)
        
        if agnostic_mode:
          box_to_color_map[box] = 'DarkOrange'
        else:
          box_to_color_map[box] = STANDARD_COLORS[
              classes[i] % len(STANDARD_COLORS)]

  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    if instance_masks is not None:
      vis_util.draw_mask_on_image_array(
          image,
          box_to_instance_masks_map[box],
          color=color
      )
    if instance_boundaries is not None:
      vis_util.draw_mask_on_image_array(
          image,
          box_to_instance_boundaries_map[box],
          color='red',
          alpha=1.0
      )
    vis_util.draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)
    if keypoints is not None:
      vis_util.draw_keypoints_on_image_array(
          image,
          box_to_keypoints_map[box],
          color=color,
          radius=line_thickness / 2,
          use_normalized_coordinates=use_normalized_coordinates)

  return image

def make_frame_log(
    boxes,
    classes,classes1,
    category_index,category_index1):
  fishermen ={}
  for i in range(boxes.shape[0]):
    class_name = category_index[classes[i]]['name']
    class_name1 = category_index1[classes1[i]]['name']
    fishermen[i] =[class_name,class_name1]
  return fishermen  
    

## Object detection imports
Here are the imports from the object detection module.

In [0]:
class FishingDetection:
    
    frame = 0
    jlog={}
    def __init__(self, human_detection_graph_path,
                 human_detection_graph_label_path,
                 num_classes,
                 fishing_classification_graph_path,
                 fishing_classification_graph_label_path,
                 gear_model_weights_path):
        self.__human_detection_graph_path = human_detection_graph_path
        self.__human_detection_graph = FishingDetection.load_graph(self.__human_detection_graph_path)
        self.__num_classes = num_classes
        self.__fishing_classification_graph_path = fishing_classification_graph_path
        self.__fishing_classification_graph = FishingDetection.load_inception(self.__fishing_classification_graph_path)
        self.__label_map = label_map_util.load_labelmap(human_detection_graph_label_path)
        self.__categories = label_map_util.convert_label_map_to_categories(self.__label_map,
                                                                           max_num_classes=self.__num_classes,
                                                                           use_display_name=True)
        self.__category_index = label_map_util.create_category_index(self.__categories)
        self.__gear_model = FishingDetection.load_trained_model_keras(gear_model_weights_path)

    @staticmethod
    def nvidia_net(drop_prob = 0.2):
      #create a sequential Model
      model = Sequential()

      #Add a Cropping layer to trim the unneeded portions of the IMAGE from the feed
      #model.add(Cropping2D)
      #model.add(Reshape((50,50,3), input_shape=(None,None,3))
      model.add(Cropping2D(cropping = ((0, 0), (0,0)), input_shape = (100 ,100 ,3)))

      #Normalization Layer
      #model.add(Lambda(lambda X_input: (X_input/255.0 - 0.5)))

      #Conv2D Layer 1 with 5 x 5 kernal size
      #model.add(Convolution2D(nb_filter = 3, nb_row = 5, nb_col = 5))
      model.add(Convolution2D(nb_filter = 12, nb_row = 3, nb_col = 3, subsample=(1,1)))
      model.add(Activation('relu'))

      #Dropout layer
      model.add(Dropout(drop_prob))

      #Conv2D Layer 2 with 5 x 5 kernal size
      #model.add(Convolution2D(nb_filter = 24, nb_row = 5, nb_col = 5))
      model.add(Conv2D(nb_filter = 24, nb_row = 3, nb_col = 3,subsample=(2,2)))
      model.add(Activation('relu'))

      #Conv2D Layer 3 with 5 x 5 kernal size
      #model.add(Convolution2D(nb_filter = 36, nb_row = 5, nb_col =  5))
      model.add(Conv2D(nb_filter = 36, nb_row = 3, nb_col =  3,subsample=(1,1)))
      model.add(Activation('relu'))

      #Dropout layer
      model.add(Dropout(drop_prob))

      #Conv2D Layer 4 with 3 x 3 kernal size
      #model.add(Convolution2D(nb_filter = 48, nb_row = 3, nb_col = 3))
      model.add(Conv2D(nb_filter = 48, nb_row = 3, nb_col = 3))
      model.add(Activation('relu'))

      #Conv2D Layer 5 with 3 x 3 kernal size
      #model.add(Convolution2D(nb_filter = 48, nb_row = 3, nb_col = 3))
      model.add(Conv2D(nb_filter = 64, nb_row = 3, nb_col = 3))
      model.add(Activation('relu'))

      #flatten layer
      #flatten the output from convolution Layer
      model.add(Flatten())

      #Fully connected layer 1
      model.add(Dense(output_dim = 50))

      #Fully connected Layer 2
      model.add(Dense(output_dim = 25))

      #Fully connected Layer 3
      model.add(Dense(output_dim = 10))

      #output Layers
      model.add(Dense(output_dim = 2, activation='softmax') )

      return model

    @staticmethod
    def load_graph(graph_path):
        detection_graph = tf.Graph()
        with detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(graph_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
        return detection_graph

    @staticmethod
    def load_inception(graph_path):
        graph = tf.Graph()
        graph_def = tf.GraphDef()
        with open(graph_path, "rb") as f:
            graph_def.ParseFromString(f.read())
        with graph.as_default():
            tf.import_graph_def(graph_def)
        return graph

    def load_image_into_numpy_array(self, image):
        (im_width, im_height) = image.size
        return np.array(image.getdata()).reshape(
            (im_height, im_width, 3)).astype(np.uint8)

    def run_inference_for_single_image(self,
                                       image,
                                       graph):
        with graph.as_default():
            with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                    'num_detections', 'detection_boxes', 'detection_scores',
                    'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                            tensor_name)
                if 'detection_masks' in tensor_dict:
                    # The following processing is only for single image
                    detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                    detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                    # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                    real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                    detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                    detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                        detection_masks, detection_boxes, image.shape[0], image.shape[1])
                    detection_masks_reframed = tf.cast(
                        tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                    # Follow the convention by adding back the batch dimension
                    tensor_dict['detection_masks'] = tf.expand_dims(
                        detection_masks_reframed, 0)
                image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

                # Run inference
                output_dict = sess.run(tensor_dict,
                                       feed_dict={image_tensor: np.expand_dims(image, 0)})

                # all outputs are float32 numpy arrays, so convert types as appropriate
                output_dict['num_detections'] = int(output_dict['num_detections'][0])
                output_dict['detection_classes'] = output_dict[
                    'detection_classes'][0].astype(np.uint8)
                output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
                output_dict['detection_scores'] = output_dict['detection_scores'][0]
                if 'detection_masks' in output_dict:
                    output_dict['detection_masks'] = output_dict['detection_masks'][0]
        return output_dict

    def list_files(self, path):
        # files = [f for f in listdir(path) if isfile(join(path, f))]
        # return files
        return glob.glob(path)

    def process_files(self,
                      list_files,
                      label):
        output = []
        for i in list_files:
            temp = []
            temp.append(i)
            ground_truth = [0, 0]
            ground_truth[label] = 1.
            temp.append(ground_truth)
            output.append(temp)
        return output

    def read_image(self, path):
        img = Image.open(path).convert('RGB')
        img = self.make_square(img)
        img = img.resize((100, 100), Image.ANTIALIAS)
        return np.asarray(img)

    def pred_keras(self,
                   img):
        # print(img)
        img = self.make_square(img)
        img = img.resize((100, 100), Image.ANTIALIAS)
        img = np.asarray(img)
        img = (img/255)-0.5
        return self.__gear_model.predict(np.expand_dims(img, axis=0))

    def make_square(self,
                    im,
                    min_size=100,
                    fill_color=(0, 0, 0, 0)):
        im = Image.fromarray(im, 'RGB')
        x, y = im.size
        size = max(min_size, x, y)
        new_im = Image.new('RGB', (size, size), fill_color)
        # print((int((size - x) / 2), int(size - y) / 2))
        new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

    @staticmethod
    def load_trained_model_keras(weights_path):
        model = FishingDetection.nvidia_net()
        model.load_weights(weights_path)
        return model

    def load_labels(self, label_file):
        label = []
        proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
        for l in proto_as_ascii_lines:
            label.append(l.rstrip())
        return label

    def read_tensor_from_image_file(self,
                                    image_np,
                                    input_height=299,
                                    input_width=299,
                                    input_mean=0,
                                    input_std=255):
        input_name = "file_reader"
        output_name = "normalized"
        file_reader = image_np
        image_reader = tf.convert_to_tensor(image_np, np.uint8)
        float_caster = tf.cast(image_reader, tf.float32)
        dims_expander = tf.expand_dims(float_caster, 0)
        resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
        normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
        sess = tf.Session()
        result = sess.run(normalized)
        return result

    def pred_tensorflow(self,
                        image_np,
                        graph,
                        input_height=299,
                        input_width=299,
                        input_mean=0,
                        input_std=255):
        sess = tf.Session(graph=graph)
        # label_file = r'D:\fishing_detection\data\output_labels.txt'
        t = self.read_tensor_from_image_file(
            image_np,
            input_height=input_height,
            input_width=input_width,
            input_mean=input_mean,
            input_std=input_std)
        input_name = "import/" + "Placeholder"
        output_name = "import/" + "final_result"
        input_operation = graph.get_operation_by_name(input_name)
        output_operation = graph.get_operation_by_name(output_name)
        with sess as sess:
            results = sess.run(output_operation.outputs[0], {
                input_operation.outputs[0]: t
            })
        results = np.squeeze(results)
        return results

    def predict_fishing(self, image, boxes, classes, scores):
        im_width, im_height = image.size
        image_np = self.load_image_into_numpy_array(image)
        predictions = {}
        predictions['bo'] = []
        predictions['res'] = []
        predictions['score'] = []
        predictions1 = {}
        predictions1['bo'] = []
        predictions1['res'] = []
        predictions1['score'] = []
        for i in range(len(boxes)):
            if classes[i] == 1 and scores[i] > 0.3:
                predictions['bo'].append(boxes[i])
                predictions1['bo'].append(boxes[i])
                box = tuple(boxes[i].tolist())
                ymin, xmin, ymax, xmax = box
                ymin = int(ymin * im_height)
                xmin = int(xmin * im_width)
                ymax = int(ymax * im_height)
                xmax = int(xmax * im_width)
                roi = image_np[ymin:ymax, xmin:xmax]
                # c = pred(cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))
                c = self.pred_tensorflow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB), self.__fishing_classification_graph)
                d = self.pred_keras(roi)[0]
                if (c[0] > 0.5):
                    predictions['res'].append(0)
                    predictions['score'].append(c[0])
                else:
                    predictions['res'].append(1)
                    predictions['score'].append(c[1])
                if (d[0] > 0.5):
                    predictions1['res'].append(0)
                    predictions1['score'].append(d[0])
                else:
                    predictions1['res'].append(1)
                    predictions1['score'].append(d[1])    
        return predictions,predictions1

    def pipeline(self, img):
        FishingDetection.frame +=1
        image = Image.fromarray(img)
        image_np = self.load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        output_dict = self.run_inference_for_single_image(image_np, self.__human_detection_graph)
        for i in range(len(output_dict['detection_classes'])):
            if output_dict['detection_classes'][i] != 1:
                output_dict['detection_scores'][i] = 0.0
        predi,predi1 = self.predict_fishing(image, output_dict['detection_boxes'], output_dict['detection_classes'],
                                     output_dict['detection_scores'])
        category_ind1 = {1: {'id': 1, 'name': 'Geared'}, 0: {'id': 0, 'name': 'Not Geared'}}
        category_ind = {1: {'id': 1, 'name': 'Not Fishing'}, 0: {'id': 0, 'name': 'Fishing'}}
        
        frame_log = make_frame_log(np.asarray(predi['bo'], dtype=np.float32),
            predi['res'],predi1['res'],
            category_ind,category_ind1,)
        
        FishingDetection.jlog[FishingDetection.frame]=frame_log
        
        visualize_boxes_and_labels_on_image_array(
            image_np,
            np.asarray(predi['bo'], dtype=np.float32),
            predi['res'],predi1['res'],
            predi['score'],predi1['score'],
            category_ind,category_ind1,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=4)
        return image_np


In [0]:
def main():
    INPUT_DIRECTORY = 'input_video'
    OUTPUT_DIRECTORY = 'output_video'
    INPUT_FILE = 'Catching_tuna_Maldivian_style.mp4'
    OUTPUT_FILE = 'result4.mp4'
    vid_output = os.path.join(OUTPUT_DIRECTORY, OUTPUT_FILE)
    vid_input = os.path.join(INPUT_DIRECTORY, INPUT_FILE)
    subclip_start = '00:00:20.50'
    subclip_end = '00:00:21.80'
    clip = VideoFileClip(vid_input).subclip(subclip_start, subclip_end)

    DIRECTORY_NAME = 'models'
    MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
    TAR_EXTENSION = '.tar.gz'
    MODEL_FILE = MODEL_NAME + TAR_EXTENSION
    GRAPH_NAME = 'frozen_inference_graph.pb'
    DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
    PATH_TO_FROZEN_GRAPH = os.path.join(DIRECTORY_NAME, MODEL_NAME, GRAPH_NAME)
    GEAR_MODEL_PATH = r'colab/model.h5'

    # List of the strings that is used to add correct label for each box.
    PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

    NUM_CLASSES = 90
    fishing_detection_graph = os.path.join('models', 'inception_retrained', 'output_graph.pb')
    fishing_classification_graph_label_path = os.path.join('data', 'output_labels.txt')
    object_detect = FishingDetection(human_detection_graph_path=PATH_TO_FROZEN_GRAPH,
                                     human_detection_graph_label_path=PATH_TO_LABELS,
                                     num_classes=NUM_CLASSES,
                                     fishing_classification_graph_path=fishing_detection_graph,
                                     fishing_classification_graph_label_path=fishing_classification_graph_label_path,
                                     gear_model_weights_path=GEAR_MODEL_PATH)
    vid = clip.fl_image(object_detect.pipeline)
    vid.write_videofile(vid_output, audio=False)
    with open('data.json', 'w') as fp:
      json.dump(FishingDetection.jlog, fp)


In [10]:
main()

predictions {'bo': [array([0.15644306, 0.17607833, 0.82251096, 0.3345195 ], dtype=float32), array([0.1137785 , 0.6466236 , 0.79842293, 0.81135726], dtype=float32), array([0.08943304, 0.3895247 , 0.8051671 , 0.5414243 ], dtype=float32), array([0.07140476, 0.8052314 , 0.7943431 , 0.9453165 ], dtype=float32), array([0.20181423, 0.00131959, 0.84252244, 0.13493222], dtype=float32)], 'res': [0, 0, 0, 0, 1], 'score': [0.9841511, 0.9456887, 0.9622137, 0.9975395, 0.60856676]}
predictions1 {'bo': [array([0.15644306, 0.17607833, 0.82251096, 0.3345195 ], dtype=float32), array([0.1137785 , 0.6466236 , 0.79842293, 0.81135726], dtype=float32), array([0.08943304, 0.3895247 , 0.8051671 , 0.5414243 ], dtype=float32), array([0.07140476, 0.8052314 , 0.7943431 , 0.9453165 ], dtype=float32), array([0.20181423, 0.00131959, 0.84252244, 0.13493222], dtype=float32)], 'res': [1, 1, 0, 1, 1], 'score': [0.99999475, 0.99999917, 0.99997294, 0.99977714, 0.9985936]}
[MoviePy] >>>> Building video output_video/result4.m

  7%|▋         | 1/14 [00:12<02:46, 12.83s/it]

predictions {'bo': [array([0.15644306, 0.17607833, 0.82251096, 0.3345195 ], dtype=float32), array([0.1137785 , 0.6466236 , 0.79842293, 0.81135726], dtype=float32), array([0.08943304, 0.3895247 , 0.8051671 , 0.5414243 ], dtype=float32), array([0.07140476, 0.8052314 , 0.7943431 , 0.9453165 ], dtype=float32), array([0.20181423, 0.00131959, 0.84252244, 0.13493222], dtype=float32)], 'res': [0, 0, 0, 0, 1], 'score': [0.9841511, 0.9456887, 0.9622137, 0.9975395, 0.60856676]}
predictions1 {'bo': [array([0.15644306, 0.17607833, 0.82251096, 0.3345195 ], dtype=float32), array([0.1137785 , 0.6466236 , 0.79842293, 0.81135726], dtype=float32), array([0.08943304, 0.3895247 , 0.8051671 , 0.5414243 ], dtype=float32), array([0.07140476, 0.8052314 , 0.7943431 , 0.9453165 ], dtype=float32), array([0.20181423, 0.00131959, 0.84252244, 0.13493222], dtype=float32)], 'res': [1, 1, 0, 1, 1], 'score': [0.99999475, 0.99999917, 0.99997294, 0.99977714, 0.9985936]}


 14%|█▍        | 2/14 [00:25<02:32, 12.74s/it]

predictions {'bo': [array([0.12592363, 0.64600253, 0.81025445, 0.8050183 ], dtype=float32), array([0.1650311 , 0.17464933, 0.84175813, 0.33380136], dtype=float32), array([0.11477658, 0.37804365, 0.8163167 , 0.5391908 ], dtype=float32), array([0.08462474, 0.80297625, 0.8086245 , 0.97007334], dtype=float32), array([0.22395694, 0.00118845, 0.865186  , 0.13015217], dtype=float32)], 'res': [0, 0, 0, 0, 0], 'score': [0.9337015, 0.9829154, 0.93259084, 0.9966145, 0.6242204]}
predictions1 {'bo': [array([0.12592363, 0.64600253, 0.81025445, 0.8050183 ], dtype=float32), array([0.1650311 , 0.17464933, 0.84175813, 0.33380136], dtype=float32), array([0.11477658, 0.37804365, 0.8163167 , 0.5391908 ], dtype=float32), array([0.08462474, 0.80297625, 0.8086245 , 0.97007334], dtype=float32), array([0.22395694, 0.00118845, 0.865186  , 0.13015217], dtype=float32)], 'res': [1, 1, 0, 1, 1], 'score': [0.99999964, 0.99999714, 0.9999932, 1.0, 0.98694175]}


...

 93%|█████████▎| 13/14 [02:48<00:12, 12.94s/it]

predictions {'bo': [array([0.1462622 , 0.14836605, 0.8841779 , 0.30356055], dtype=float32), array([0.2275961, 0.       , 0.8915234, 0.1018519], dtype=float32), array([0.0429613 , 0.34485924, 0.8991736 , 0.5017943 ], dtype=float32), array([0.15634248, 0.82664585, 0.89921916, 0.9868779 ], dtype=float32), array([0.15696895, 0.6273671 , 0.8929466 , 0.7724188 ], dtype=float32)], 'res': [0, 0, 0, 0, 0], 'score': [0.98630303, 0.92662895, 0.9922363, 0.9532817, 0.9660247]}
predictions1 {'bo': [array([0.1462622 , 0.14836605, 0.8841779 , 0.30356055], dtype=float32), array([0.2275961, 0.       , 0.8915234, 0.1018519], dtype=float32), array([0.0429613 , 0.34485924, 0.8991736 , 0.5017943 ], dtype=float32), array([0.15634248, 0.82664585, 0.89921916, 0.9868779 ], dtype=float32), array([0.15696895, 0.6273671 , 0.8929466 , 0.7724188 ], dtype=float32)], 'res': [1, 1, 0, 1, 1], 'score': [0.99978465, 0.98946744, 0.9999603, 1.0, 0.9999362]}


100%|██████████| 14/14 [03:01<00:00, 12.99s/it]

predictions {'bo': [array([0.1886864 , 0.13969089, 0.9212098 , 0.29736328], dtype=float32), array([0.19120696, 0.61855996, 0.92342496, 0.761408  ], dtype=float32), array([0.20166466, 0.80411077, 0.9482944 , 0.9900826 ], dtype=float32), array([0.23629364, 0.        , 0.92610884, 0.09415025], dtype=float32), array([0.06584823, 0.34288946, 0.93276626, 0.5005456 ], dtype=float32)], 'res': [0, 0, 0, 0, 0], 'score': [0.9906729, 0.9895154, 0.9348564, 0.95831996, 0.9986112]}
predictions1 {'bo': [array([0.1886864 , 0.13969089, 0.9212098 , 0.29736328], dtype=float32), array([0.19120696, 0.61855996, 0.92342496, 0.761408  ], dtype=float32), array([0.20166466, 0.80411077, 0.9482944 , 0.9900826 ], dtype=float32), array([0.23629364, 0.        , 0.92610884, 0.09415025], dtype=float32), array([0.06584823, 0.34288946, 0.93276626, 0.5005456 ], dtype=float32)], 'res': [1, 1, 1, 1, 0], 'score': [0.99992836, 0.9999654, 1.0, 0.923555, 0.9976301]}


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video/result4.mp4 



In [0]:
from google.colab import files
files.download('data.json') 